importing libraries

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import (accuracy_score,confusion_matrix,classification_report,roc_curve,auc,precision_recall_curve)
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

import shap

pd.set_option("display.max_columns",None)

c:\Users\sahil kavishka\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load dataset

In [15]:
matches=pd.read_csv("all_champions_trophy_matches_results.csv")
players=pd.read_csv("all_champions_trophy_players_list.csv")

matches.head(),players.head()

(          Team1        Team2                                  Toss Match days  \
 0  South Africa  West Indies  South Africa, elected to field first   daynight   
 1         India  West Indies           India, elected to bat first   daynight   
 2  South Africa    Sri Lanka     Sri Lanka, elected to field first   daynight   
 3      Pakistan  West Indies     West Indies, elected to bat first   daynight   
 4     Australia        India     Australia, elected to field first   daynight   
 
          Winner Player of the Match     Margin Ground    Match Date  \
 0  South Africa      Jacques Kallis  4 wickets  Dhaka   Nov 1, 1998   
 1   West Indies       Mervyn Dillon  6 wickets  Dhaka  Oct 31, 1998   
 2  South Africa      Jacques Kallis    92 runs  Dhaka  Oct 30, 1998   
 3   West Indies     Keith Arthurton    30 runs  Dhaka  Oct 29, 1998   
 4         India    Sachin Tendulkar    44 runs  Dhaka  Oct 28, 1998   
 
   ODI Int Match  Team1 Avg Batting Ranking  Team2 Avg Batting Ranking  

data cleaning

In [18]:
#strip column name
matches.columns=matches.columns.str.strip()
players.columns=players.columns.str.strip()

#drop duplicates
matches.drop_duplicates(inplace=True)

#drop rows with missing value
matches=matches.dropna(subset=["Winner","Team1","Team2"])

#clean string fields
for col in ["Winner","Team1","Team2","Ground","Margin","Match Date","Toss","Match days"]:
    if col in matches.columns:
        matches[col]=matches[col].astype(str).str.strip()

#convert match date
matches["Date"]=pd.to_datetime(matches["Match Date"],format="%b %d, %y",errors="coerce")

matches.head()


,Team1,Team2,Toss,Match days,Winner,Player of the Match,Margin,Ground,Match Date,ODI Int Match,Team1 Avg Batting Ranking,Team2 Avg Batting Ranking,Team1 Avg Bowling Ranking,Team2 Avg Bowling Ranking,Team1 Total CTs participated,Team1 Total CTs won,Team2 Total CTs participated,Team2 Total CTs won,Team1 W/L ratio over Team2,Date
0,South Africa,West Indies,"South Africa, elected to field first",daynight,South Africa,Jacques Kallis,4 wickets,Dhaka,"Nov 1, 1998",ODI # 1364,66.58,63.42,81.50,84.92,0,0,0,0,1.000,NaT
1,India,West Indies,"India, elected to bat first",daynight,West Indies,Mervyn Dillon,6 wickets,Dhaka,"Oct 31, 1998",ODI # 1363,58.00,63.92,63.73,86.25,0,0,0,0,0.527,NaT
2,South Africa,Sri Lanka,"Sri Lanka, elected to field first",daynight,South Africa,Jacques Kallis,92 runs,Dhaka,"Oct 30, 1998",ODI # 1362,66.92,53.55,83.08,69.18,0,0,0,0,1.166,NaT
3,Pakistan,West Indies,"West Indies, elected to bat first",daynight,West Indies,Keith Arthurton,30 runs,Dhaka,"Oct 29, 1998",ODI # 1361,88.36,65.00,70.00,87.17,0,0,0,0,0.806,NaT
4,Australia,India,"Australia, elected to field first",daynight,India,Sachin Tendulkar,44 runs,Dhaka,"Oct 28, 1998",ODI # 1360,58.55,58.00,78.73,64.82,0,0,0,0,1.380,NaT


fuature engineering